# Perform Sentiment Analysis in Python 3 Using the Natural Language Toolkit (NLTK)

Click [here](https://www.digitalocean.com/community/tutorials/how-to-perform-sentiment-analysis-in-python-3-using-the-natural-language-toolkit-nltk) for source.

## Import Modules

In [117]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier

import re, string, random
from pprint import pprint

## Define Functions

In [118]:
# remove noise, normalize and lemmatize tweet text
def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

# provide a list of words in all of the tweet tokens
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

# convert the tweets from a list of cleaned tokens to dictionaries with keys as the tokens and True as values            
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

## Import and Tokenize Tweets

In [119]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')
#tweet_tokens = twitter_samples.tokenized('positive_tweets.json')[0]

stop_words = stopwords.words('english')

positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

## Normalize, Lemmatize and Clean Tokens of Stop Words

In [120]:
positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

## Determine Word Density

In [121]:
all_pos_words = get_all_words(positive_cleaned_tokens_list)

freq_dist_pos = FreqDist(all_pos_words)

# list the 10 most common positive words
pprint(freq_dist_pos.most_common(10))

[(':)', 3691),
 (':-)', 701),
 (':d', 658),
 ('thanks', 388),
 ('follow', 357),
 ('love', 333),
 ('...', 290),
 ('good', 283),
 ('get', 263),
 ('thank', 253)]


## Prepare Data for the Model

#### Convert tokens to dictionary

In [122]:
positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

#### Split data into training and test datasets

In [123]:
positive_dataset = [(tweet_dict, "Positive")
                     for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in negative_tokens_for_model]

dataset = positive_dataset + negative_dataset

random.seed(456) # set seed for reproducibility
random.shuffle(dataset)

train_data = dataset[:7000]
test_data = dataset[7000:]

## Build and Test the Model

#### Build the Model

In [124]:
classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))

print(classifier.show_most_informative_features(10))

Accuracy is: 0.9963333333333333
Most Informative Features
                      :( = True           Negati : Positi =   2067.5 : 1.0
                      :) = True           Positi : Negati =   1655.0 : 1.0
                     sad = True           Negati : Positi =     36.1 : 1.0
                     bam = True           Positi : Negati =     23.5 : 1.0
                 welcome = True           Positi : Negati =     22.0 : 1.0
                follower = True           Positi : Negati =     21.6 : 1.0
                   enjoy = True           Positi : Negati =     20.8 : 1.0
                     x15 = True           Negati : Positi =     20.5 : 1.0
                    glad = True           Positi : Negati =     13.3 : 1.0
                    blog = True           Positi : Negati =     12.9 : 1.0
None


#### Test the Model

In [125]:
# Test Negative Tweet
custom_tweet = "I ordered just once from TerribleCo, they screwed up, never used the app again."
custom_tokens = remove_noise(word_tokenize(custom_tweet))
print(custom_tweet, '\n', classifier.classify(dict([token, True] for token in custom_tokens)), '\n')

# Test Positive Tweet
custom_tweet = "Congrats #SportStar on your 7th best goal from last season winning goal of the year :) #Baller #Topbin #oneofmanyworldies"
custom_tokens = remove_noise(word_tokenize(custom_tweet))
print(custom_tweet, '\n', classifier.classify(dict([token, True] for token in custom_tokens)), '\n')

# Test Sarcastic Tweet
custom_tweet = "Thank you for sending my baggage to CityX and flying me to CityY at the same time. Brilliant service. #thanksGenericAirline"
custom_tokens = remove_noise(word_tokenize(custom_tweet))
print(custom_tweet, '\n', classifier.classify(dict([token, True] for token in custom_tokens)))

I ordered just once from TerribleCo, they screwed up, never used the app again. 
 Negative 

Congrats #SportStar on your 7th best goal from last season winning goal of the year :) #Baller #Topbin #oneofmanyworldies 
 Positive 

Thank you for sending my baggage to CityX and flying me to CityY at the same time. Brilliant service. #thanksGenericAirline 
 Positive
